In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from broverse.action import Action
from broverse.flow import Flow
from broverse.bedrock import BedrockChat
from typing import Any
import yaml

In [3]:
class InputAction(Action):
    def run(self, shared):
        message = input("You: ")
        shared['input'] = message
        return message

    def validate_next_action(self, inputs: Any) -> str:
        if "exit" == inputs.lower():
            return "end"
        return "router"
    
class RouterAction(Action):
    def __init__(self, system_prompt:str, model:BedrockChat):
        super().__init__()
        self.system_prompt = system_prompt
        self.model = model

    def run(self, shared):
        input_message = shared.get("input", "No message")
        intent = self.model.run(self.system_prompt, [self.model.UserMessage(text=input_message)])
        intent = intent.split("```yaml")[1].split("```")[0].strip()
        intent = yaml.safe_load(intent)['action']
        print("Intent:", intent)
        return intent
    
    def validate_next_action(self, inputs:Any) -> str:
        if inputs == 'farewell':
            return 'farewell'
        return "chat"

class ChatAction(Action):
    def __init__(self, system_prompt:str, model:BedrockChat):
        super().__init__()
        self.system_prompt = system_prompt
        self.model = model
    
    def run(self, shared):
        if "messages" not in shared:
            shared["messages"] = []
        input_message = shared.get("input", "No message")
        shared["messages"].append(self.model.UserMessage(text=input_message))
        ai_response = self.model.run(self.system_prompt, shared["messages"])
        shared["messages"].append(self.model.AIMessage(text=ai_response))
        print("AI:", ai_response)
        return ai_response
    
    def validate_next_action(self, inputs: Any) -> str:
        return "continue"
    
class FarewellAction(Action):
    def __init__(self, system_prompt:str, model:BedrockChat):
        super().__init__()
        self.system_prompt = system_prompt
        self.model = model    

    def run(self, shared):
        if "messages" not in shared:
            shared["messages"] = []
        input_message = shared.get("input", "No message")
        shared["messages"].append(self.model.UserMessage(text=f"{input_message}\n\nI gotta go now. See ya next time."))
        ai_response = self.model.run(self.system_prompt, shared["messages"])
        shared["messages"].append(self.model.AIMessage(text=ai_response))
        print("AI:", ai_response)
        return ai_response
    
    def validate_next_action(self, inputs: Any) -> str:
        return "end"

class End(Action):
    def run(self, shared):
        return None

router_prompt = """\
classify a user's intent based on the input messages. 
Intent options are:
1. continue if nothing goes wrong
2. farewell if a user's message indicate that he or she wants to go somewhere

Return your response in codeblock with this following yaml format:
```yaml
action: either continue or farewell
```

IMPORTANT: Make sure to:
1. Use proper indentation (4 spaces) for all multi-line fields
2. Use the | character for multi-line text fields
3. Keep single-line fields without the | character
""".strip()

In [4]:
input_action = InputAction()
chat_action = ChatAction(
    system_prompt="You are a helpful assistant",
    model=BedrockChat()
)
router_action = RouterAction(
    system_prompt=router_prompt,
    model=BedrockChat()
)

farewell_action = FarewellAction(
    system_prompt="Your job is to farewell a user.",
    model=BedrockChat()
)

In [5]:
input_action -"router">> router_action
router_action -"chat">> chat_action
router_action -"farewell">> farewell_action
chat_action -"continue">> input_action

for action in [input_action, farewell_action]:
    action -"end">> End()


In [6]:
flow = Flow(start_action=input_action)
flow.save_mermaid(filename="flow_chart.md")

In [7]:
chart = flow.to_mermaid()
print(chart)

```mermaid
flowchart TD
    InputAction -->|router| RouterAction
    RouterAction -->|chat| ChatAction
    ChatAction -->|continue| InputAction
    RouterAction -->|farewell| FarewellAction
    FarewellAction -->|end| End
    InputAction -->|end| End
```


In [8]:
shared = {}
flow.run(shared=shared)

Intent: farewell
AI: It was nice chatting with you. Have a great day and I'll see you next time!


'default'

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(["text1", "text2"])  # Done!


d:\broverse\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
len(embeddings[0]), embeddings.shape, embeddings.shape[1]

(384, (2, 384), 384)

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
import boto3
dynamodb = boto3.resource('dynamodb', region_name='ap-southeast-1')
# table = dynamodb.Table(table_name)

In [6]:
table = dynamodb.Table("test-broverse")

In [17]:
from broverse.program.dynamodb import create_document, update_document_status, delete_document

In [15]:
create_document(
    user_id="000",
    document_id="555",
    username="admin",
    file_path="test-file",
    vector_path="test-vector",
    status="CREATED"
)

In [16]:
update_document_status(user_id="000", document_id="555", new_status="READY")

In [18]:
delete_document(user_id="000", document_id="555")

In [19]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(["text1", "text2"])  # Done!

d:\broverse\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
texts = [
    "Hello there",
    "I'm hungry",
    "Oh my god, are you ok?"
]

vectors = model.encode(texts)

In [35]:
embedding_vectors = []
enum = 0
for t, v in zip(texts, vectors):
    obj = {"key": f"v{enum}", "data": {"float32": v.tolist()}, "metadata": {"id": f"key{enum}", "source_text": t, "source": "url"}}
    embedding_vectors.append(obj)
    enum += 1

In [36]:
import boto3

# Create S3Vectors client
s3vectors = boto3.client('s3vectors', region_name='us-west-2')

# Insert vector embedding
s3vectors.put_vectors( vectorBucketName="test-broverse-20250729",
  indexName="test-url-index", 
  vectors=embedding_vectors,
)

{'ResponseMetadata': {'RequestId': 'fcc16795-a2fa-463f-bede-28901d6d5b73',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 28 Jul 2025 17:25:53 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amz-request-id': 'fcc16795-a2fa-463f-bede-28901d6d5b73',
   'access-control-allow-origin': '*',
   'vary': 'origin, access-control-request-method, access-control-request-headers',
   'access-control-expose-headers': '*'},
  'RetryAttempts': 0}}

In [37]:
input_query = "I'm straving"
vector = model.encode([input_query])[0].tolist()

In [40]:
# Performa a similarity query. You can also optionally use a filter in your query
query = s3vectors.query_vectors( vectorBucketName="test-broverse-20250729",
  indexName="test-url-index",
  queryVector={"float32":vector},
  topK=3, 
  filter={"source":"url"},
  returnDistance=True,
  returnMetadata=True
  )
results = query["vectors"]
print(results)

[{'key': 'v1', 'metadata': {'source': 'url', 'source_text': "I'm hungry", 'id': 'key1'}, 'distance': 0.5873361825942993}, {'key': 'v2', 'metadata': {'source_text': 'Oh my god, are you ok?', 'source': 'url', 'id': 'key2'}, 'distance': 0.7358927130699158}, {'key': 'v0', 'metadata': {'source_text': 'Hello there', 'source': 'url', 'id': 'key0'}, 'distance': 0.777753472328186}]


In [44]:
from broverse.program.s3vector import save_to_s3_vectors, query_from_s3_vectors
from broverse.interface import Context

In [45]:
contexts = [
    Context(context="I'm straving", metadata={"source": "url"}),
    Context(context="I see death people", metadata={"source": "url"}),
    Context(context="Those puppies are cute", metadata={"source": "url"}),
]

vectors = model.encode([context.context for context in contexts])
save_to_s3_vectors(contexts, vectors)

{'ResponseMetadata': {'RequestId': '0a284fc9-d3a3-4400-87b3-1291c145ea6e',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 28 Jul 2025 17:58:10 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amz-request-id': '0a284fc9-d3a3-4400-87b3-1291c145ea6e',
   'access-control-allow-origin': '*',
   'vary': 'origin, access-control-request-method, access-control-request-headers',
   'access-control-expose-headers': '*'},
  'RetryAttempts': 0}}

In [12]:
from broverse.program.s3vector import save_to_s3_vectors, query_from_s3_vectors
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(["text1", "text2"])  # Done!

user_query = "What is agentic AI?"
vector = model.encode([user_query])[0].tolist()
response = query_from_s3_vectors(vector)

In [13]:
response

[Context(context='and provide comprehensive solutions for AI workloads.\n\n**Create Amazon Bedrock Knowledge Bases with S3 Vectors**\n\n You can use S3 Vectors in Amazon Bedrock Knowledge Bases to simplify and reduce the cost of vector storage for RAG applications. When creating a knowledge base in the Amazon Bedrock console, you can choose the S3 vector bucket as your vector store option.\n\nIn **Step 3**, you can choose the **Vector store creation method** either to create an S3 vector bucket and vector index or choose the existing S3 vector bucket and vector index that you’ve previously created.\n\n\n\nFor detailed step-by-step instructions, visit Create a knowledge base by connecting to a data source in Amazon Bedrock Knowledge Basesin the Amazon Bedrock User Guide.\n\n**Using Amazon SageMaker Unified Studio**You can create and manage knowledge bases with S3 Vectors in Amazon SageMaker Unified Studio when you build your generative AI applications through Amazon Bedrock. SageMaker U

In [1]:
from broverse.flows.offlineflow import get_offline_flow

flow = get_offline_flow()

d:\broverse\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
shared = {
    "user_id": "000",
    "username": "admin"
}
flow.run(shared)

End of OfflineIndex Flow


In [3]:
shared.keys()

dict_keys(['user_id', 'username', 'document_type', 'source', 'action', 'document_id', 'raw_context', 'contexts', 'vectors'])

In [4]:
shared["vectors"].shape

(16, 384)

In [5]:
len(shared["contexts"])

16

In [6]:
shared["contexts"]

[Context(context='Title: Introducing Amazon S3 Vectors: First cloud storage with native vector support at scale (preview) | Amazon Web Services\n\nURL Source: https://aws.amazon.com/blogs/aws/introducing-amazon-s3-vectors-first-cloud-storage-with-native-vector-support-at-scale/\n\nPublished Time: 2025-07-15T16:33:32-07:00\n\nMarkdown Content:\n![Image 1: Voiced by Polly](https://aws.amazon.com/polly/)\n\nToday, we’re announcing the preview of Amazon S3 Vectors, a purpose-built durable vector storage solution that can reduce the total cost of uploading, storing, and querying vectors by up to 90 percent. Amazon S3 Vectors is the first cloud object store with native support to store large vector datasets and provide subsecond query performance that makes it affordable for businesses to store AI-ready data at massive scale.\n\nVector search is an emerging technique used in generative AI applications to find similar data points to given data by comparing their vector representations using d

In [7]:
shared["raw_context"]

'Title: Introducing Amazon S3 Vectors: First cloud storage with native vector support at scale (preview) | Amazon Web Services\n\nURL Source: https://aws.amazon.com/blogs/aws/introducing-amazon-s3-vectors-first-cloud-storage-with-native-vector-support-at-scale/\n\nPublished Time: 2025-07-15T16:33:32-07:00\n\nMarkdown Content:\n![Image 1: Voiced by Polly](https://aws.amazon.com/polly/)\n\nToday, we’re announcing the preview of Amazon S3 Vectors, a purpose-built durable vector storage solution that can reduce the total cost of uploading, storing, and querying vectors by up to 90 percent. Amazon S3 Vectors is the first cloud object store with native support to store large vector datasets and provide subsecond query performance that makes it affordable for businesses to store AI-ready data at massive scale.\n\nVector search is an emerging technique used in generative AI applications to find similar data points to given data by comparing their vector representations using distance or simila

In [8]:
shared["source"]

'https://aws.amazon.com/blogs/aws/introducing-amazon-s3-vectors-first-cloud-storage-with-native-vector-support-at-scale/'

In [9]:
shared['action']

In [ ]:
import boto3

s3vectors = boto3.client('s3vectors', region_name='us-west-2')
response = s3vectors.delete_index(
    vectorBucketName='test-broverse-20250729',
    indexName='test-url-index',
    # indexArn=''
)

In [2]:
from broverse.flows.onlineflow import get_online_flow

shared = {}

flow = get_online_flow()
flow.run(shared=shared)

You: Hi
USER INTENT: continue
AI: How can I assist you today?
You: exit
End of Online Flow


In [2]:
shared

{'user_input': '',
 'action': 'chat',
 'messages': [{'role': 'user', 'content': [{'text': 'Hi there!'}]},
  {'role': 'assistant',
   'content': [{'text': "It's nice to meet you. Is there something I can help you with or would you like to chat?"}]},
  {'role': 'user', 'content': [{'text': 'How ya been?'}]},
  {'role': 'assistant',
   'content': [{'text': "I've been doing well, thanks for asking. I'm a large language model, so I don't have feelings or experiences like humans do, but I'm always happy to chat and help with any questions or topics you'd like to discuss. How about you? How's your day going so far?"}]},
  {'role': 'user', 'content': [{'text': 'Good Good'}]},
  {'role': 'assistant',
   'content': [{'text': "It's great to hear that. If you're feeling good, I'm happy to keep the conversation going. We could talk about your interests, hobbies, or anything else that's on your mind. Or if you're feeling relaxed, we could just chat about random things and see where the conversation 